In [6]:
import numpy as np
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Objective Function: Evaluate feature subsets
def fitness_function(features, X_train, X_test, y_train, y_test):
    selected_features = np.where(features > 0.5)[0]  # Threshold to select features
    if len(selected_features) == 0:  # Avoid empty subsets
        return 0
    X_train_subset = X_train[:, selected_features]
    X_test_subset = X_test[:, selected_features]
    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(X_train_subset, y_train)
    y_pred = model.predict(X_test_subset)
    return accuracy_score(y_test, y_pred)

# GWO Algorithm
def grey_wolf_optimizer(fitness_func, X_train, X_test, y_train, y_test, n_wolves=5, n_iterations=20):
    n_features = X_train.shape[1]
    wolves = np.random.rand(n_wolves, n_features)  # Initialize random wolves
    alpha, beta, delta = np.zeros(n_features), np.zeros(n_features), np.zeros(n_features)
    alpha_score, beta_score, delta_score = -np.inf, -np.inf, -np.inf

    for iter in range(n_iterations):
        for i in range(n_wolves):
            fitness = fitness_func(wolves[i], X_train, X_test, y_train, y_test)
            if fitness > alpha_score:
                alpha, beta, delta = wolves[i], alpha, beta
                alpha_score, beta_score, delta_score = fitness, alpha_score, beta_score
            elif fitness > beta_score:
                beta, delta = wolves[i], beta
                beta_score, delta_score = fitness, beta_score
            elif fitness > delta_score:
                delta = wolves[i]
                delta_score = fitness

        # Update positions
        a = 2 - 2 * (iter / n_iterations)
        for i in range(n_wolves):
            for j in range(n_features):
                r1, r2 = np.random.rand(), np.random.rand()
                A1, C1 = 2 * a * r1 - a, 2 * r2
                D_alpha = abs(C1 * alpha[j] - wolves[i, j])
                X1 = alpha[j] - A1 * D_alpha

                r1, r2 = np.random.rand(), np.random.rand()
                A2, C2 = 2 * a * r1 - a, 2 * r2
                D_beta = abs(C2 * beta[j] - wolves[i, j])
                X2 = beta[j] - A2 * D_beta

                r1, r2 = np.random.rand(), np.random.rand()
                A3, C3 = 2 * a * r1 - a, 2 * r2
                D_delta = abs(C3 * delta[j] - wolves[i, j])
                X3 = delta[j] - A3 * D_delta

                wolves[i, j] = (X1 + X2 + X3) / 3

    return alpha, alpha_score

# Load Dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run GWO for Feature Selection
best_features, best_score = grey_wolf_optimizer(fitness_function, X_train, X_test, y_train, y_test)
selected_features = np.where(best_features > 0.5)[0]
print("Selected Features:", selected_features)
print("Best Score:", best_score)


Selected Features: [0]
Best Score: 1.0
